<a href="https://colab.research.google.com/github/La-matrice/brouhaha/blob/main/brouhaha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Exécution du modèle 'brouhaha' entrainé**
**1) Identification des mots, évaluation et attribution des formes probables**

*   Input : début d'une phrase saisie par l'utilisateur
*   Output : 'logs.txt'


**2) Prédiction de la forme probable suivante**

*   Input : 'logs.txt'
*   Output : 'logs.txt'

**3) Affichage du mot probable suivant**

*   Input : 'logs.txt'
*   Output : phrase saisie par l'utilisateur complétée

# **1) Identification des mots, évaluation et attribution des formes probables**

In [33]:
import pandas as pd
import csv

def processus_phrases(phrase_input):

    print("Début de l'identification...")

    plafond = 150

    dictionnaire = pd.read_csv('/content/drive/MyDrive/brouhaha_save/dictionnaire.txt', sep='|', header=None, skiprows=1, encoding='utf-8')
    dictionnaire[0] = dictionnaire[0].apply(lambda x: x.lower())
    phrases = pd.read_csv("/content/drive/MyDrive/brouhaha_save/mots.txt", delimiter = "\t", header=None)
    phrase_input = phrase_input.lower().strip()
    k = len(phrase_input) + 1
    mots_input = []
    affichage_phrase = ""

    # boucler jusqu'à trouver toutes les expressions
    while k > 0:
        found = False
        i = min(27, k)  # 27 est le nombre maximum de caractères d'une expression du dictionnaire

        while not found:
            match_index = dictionnaire[dictionnaire[0] == phrase_input[:i]].index.tolist()

            if match_index:
                found = True
                mot = dictionnaire.loc[match_index[0], 0]
                affichage_phrase = affichage_phrase + " " + mot
                id_ = dictionnaire.loc[match_index[0], 1]
                cv = dictionnaire.loc[match_index[0], 2]
                freq = dictionnaire.loc[match_index[0], 6]
                mots_input.append([mot, id_, '', '', '', '', cv, freq])

                # décrémenter phrase_input
                if ' ' in phrase_input or "'" in phrase_input:
                    phrase_input = phrase_input[len(mot):].strip()
                    k = len(phrase_input)
                else:
                    phrase_input = ''
                    k = 0
            else:
                i -= 1

            # si aucune expression n'est trouvée, arrêter le programme
            if i == 0:
                print(f"Certains mots sont introuvables dans le dictionnaire : {phrase_input}")
                k = 0
                break

    affichage_phrase = affichage_phrase.strip()
    print(affichage_phrase)
    print("Identification terminée !")

    # évaluer la forme probable de chaque expression trouvée
    if found == True and i > 0:
        print("Début de l'évaluation des formes...")
        en_cours = pd.DataFrame(mots_input, columns=['Mot', 'ID', 'ID principal probable', '% ID principal', 'ID précis probable', '% ID précis probable', 'CV', 'Fréquence'])
        limite = len(mots_input)

        # tester chaque mot
        for k in range(limite):
            j = 1
            pourcentage_forme_principale = 0
            portion = ""
            best_score = 0
            formes_probables_principales = []
            formes_probables = []

            # si le mot possède plusieurs formes, évaluer sa forme
            if "&" in mots_input[k][1]:
                print("Le mot suivant possède plusieurs formes :",mots_input[k][0])

                if limite == 1:
                    print(f"Le mot {mots_input[k][0]} comporte plusieurs formes, merci de saisir plus d'un mot pour lancer la génération !")
                    exit()

                # tester 4 positions différentes (de n+1, n+2 à n-1, n-2) et identifier le mot indice
                for position in range(4):
                    compteur = 0

                    if position < 2:
                        if k + (position + 1) < limite:
                            indice = mots_input[k + (position + 1)][0]
                        else:
                            continue
                    else:
                        if k - (position - 1) > - 1:
                            indice = mots_input[k - (position - 1)][0]
                        else:
                            continue

                    # rechercher uniquement les lignes qui contiennent l'indice
                    lignes_phrases = phrases[phrases[0].str.contains(indice)]

                    # tester chaque ligne du fichier 'Mots'
                    for index, row in lignes_phrases.head(plafond).iterrows():
                        ligne_phrase = row[0]
                        ligne_phrase = pd.DataFrame(ligne_phrase.strip('|').split('|'))
                        contains_indice = ligne_phrase[0].str.contains(indice)
                        positions_indice = contains_indice.where(contains_indice==True).dropna().index.tolist()
                        z = ligne_phrase.shape[0]

                        for position_indice in positions_indice:
                            if ligne_phrase[0][position_indice] == indice:
                                x = position_indice

                                # relever la portion de la ligne correspondante
                                if position <= 1:
                                    if x - (position + 1) > 0:
                                        portion = ligne_phrase[0][x - (position + 1)]
                                    else:
                                        break
                                else:
                                    if x + (position - 1) < z:
                                        portion = ligne_phrase[0][x + (position - 1)]
                                    else:
                                        break

                                # rechercher la ligne du dictionnaire qui contient la portion
                                lignes_dictionnaire = dictionnaire[dictionnaire[0] == portion]
                                formes_dictionnaire = []
                                formes_dictionnaire = lignes_dictionnaire[1].str.split(' & ').explode().tolist()
                                m = 0
                                n = len(formes_dictionnaire)
                                liste_doublons = []

                                # saisir les formes probables principales et formes probables précises ainsi que leur nombre d'occurrence
                                for forme_dictionnaire in formes_dictionnaire:
                                    compteur += 1
                                    forme = forme_dictionnaire
                                    forme_principale = forme[:3]
                                    doublon = forme_principale in liste_doublons

                                    # sasir la forme complète (forme la plus précise)
                                    for forme_probable in formes_probables:
                                        if forme == forme_probable[0]:
                                            forme_probable[position + 1] += 1
                                            break
                                    else:
                                        ligne = ["", 0, 0, 0, 0]
                                        ligne[0] = forme
                                        ligne[position + 1] += 1
                                        formes_probables.append(ligne)

                                    # saisir la forme principale
                                    # vérifier qu'il n'y a pas de doublon de forme principale pour cette même expression
                                    if not doublon:
                                        liste_doublons.append(forme_principale)

                                        # si pas de doublon saisir le nombre d'occurrence
                                        for forme_probable_principale in formes_probables_principales:
                                            if forme_principale == forme_probable_principale[0]:
                                                forme_probable_principale[position + 1] += 1
                                                break
                                        else:
                                            ligne = ["", 0, 0, 0, 0]
                                            ligne[0] = forme_principale
                                            ligne[position + 1] += 1
                                            formes_probables_principales.append(ligne)
                                    m += 1

                # calculer la moyenne des positions par forme principale
                for position in range(1, 5):
                    compteur = 0
                    pourcentage = 0
                    i = 0

                    while i < len(formes_probables_principales) and formes_probables_principales[i][0] != "":
                        compteur += 1

                        if formes_probables_principales[i][position] > 0:
                            pourcentage += formes_probables_principales[i][position]
                        i += 1

                    # saisir le pourcentage de présence d'une forme pour chaque position
                    for i in range(compteur):
                        if formes_probables_principales[i][position] > 0:
                            formes_probables_principales[i][position] = (formes_probables_principales[i][position] * 100) / pourcentage

                # saisir la moyenne
                for i in range(compteur):
                    pourcentage = 0

                    for position in range(1, 5):
                        pourcentage += formes_probables_principales[i][position]
                    formes_probables_principales[i][1] = pourcentage / 4

                # trier les formes_probables_principales par ordre décroissant des moyennes
                formes_probables_principales.sort(key=lambda x: x[1], reverse=True)

                # attribuer la forme principale la plus probable
                compteur = mots_input[k][1].count("&")
                pourcentage = 0
                best_forme = 0

                for i in range(compteur + 1):
                    j = 0

                    while j < len(formes_probables_principales):
                        id_unique = mots_input[k][1].split("&")[i].strip()[:3]  # 3 pour uniquement connaitre la forme principale

                        if id_unique == formes_probables_principales[j][0]:
                            if pourcentage < formes_probables_principales[j][1]:
                                best_forme = mots_input[k][1].split("&")[i].strip()
                                pourcentage = formes_probables_principales[j][1]
                                #c = pourcentage
                                mots_input[k][2] = formes_probables_principales[j][0]
                                mots_input[k][3] = pourcentage
                                pourcentage_forme_principale = pourcentage
                        j += 1

                # si aucune forme similaire trouvée du mot input en cours, attribution de la forme principale la plus probable
                if best_forme == 0:
                    print("Aucune forme principale similaire trouvée...")
                    mots_input[k][2] = formes_probables_principales[0][0]
                    mots_input[k][3] = formes_probables_principales[0][1]

                # calculer la moyenne des positions par forme précise à partir de la forme principale la plus probable
                for position in range(1, 5):
                    compteur = 0
                    pourcentage = 0
                    i = 0

                    while i < len(formes_probables):
                        compteur += 1

                        if formes_probables[i][0][:3] == mots_input[k][2]:
                            if formes_probables[i][position] > 0:
                                pourcentage += formes_probables[i][position]
                        else:
                            for p in range(1, 5):
                                formes_probables[i][p] = 0
                        i += 1

                    # saisir le pourcentage de présence d'une forme pour chaque position
                    for i in range(compteur):
                        if formes_probables[i][position] > 0:
                            formes_probables[i][position] = (formes_probables[i][position] * 100) / pourcentage

                # saisir la moyenne
                for i in range(compteur):
                    pourcentage = 0

                    for position in range(1, 5):
                        pourcentage += formes_probables[i][position]
                    formes_probables[i][1] = pourcentage / 4

                # trier les formes probables par ordre décroissant des moyennes
                formes_probables.sort(key=lambda x: x[1], reverse=True)

                # attribuer la forme la plus probable
                compteur = mots_input[k][1].count("&")
                pourcentage = 0

                for i in range(compteur + 1):
                    j = 0

                    while j < len(formes_probables) and formes_probables[j][0] != "":
                        id_unique = mots_input[k][1].split("&")[i].strip()

                        if id_unique == formes_probables[j][0]:
                            if pourcentage < formes_probables[j][1]:
                                best_forme = mots_input[k][1].split("&")[i].strip()
                                best_score = formes_probables[j][1]
                                pourcentage = formes_probables[j][1]
                        j += 1

                mots_input[k][4] = best_forme
                mots_input[k][5] = best_score

                # si aucune forme similaire trouvée du mot input en cours, attribution de la forme précise la plus probable
                if best_forme == 0:
                    print("Aucune forme précise similaire trouvée...")
                    mots_input[k][4] = formes_probables[0][0]
                    mots_input[k][5] = formes_probables[0][1]

                print("   Nouvelle forme attribuée : ", mots_input[k][4])

            # si le mot dispose que d'une seule forme
            else:
                mots_input[k][2] = mots_input[k][1][:3]
                mots_input[k][3] = "100"
                mots_input[k][4] = mots_input[k][1]
                mots_input[k][5] = "100"
                mots_input[k][6] = cv
                mots_input[k][7] = freq

        with open('logs.txt', "w", newline="") as fichier:
                writer = csv.writer(fichier, delimiter='|')
                writer.writerows(mots_input)

if __name__ == '__main__':
    processus_phrases("il ne pense pas que")

Début de l'identification...
il ne pense pas que
Identification terminée !
Début de l'évaluation des formes...
Le mot suivant possède plusieurs formes : pense
   Nouvelle forme attribuée :  VER:P0:T14:ind:pre:1s
Le mot suivant possède plusieurs formes : pas
   Nouvelle forme attribuée :  ADV:neg:pas
Le mot suivant possède plusieurs formes : que
   Nouvelle forme attribuée :  PRO:int:cho


#**2) Prédiction de la forme probable suivante**

In [35]:
import pandas as pd
import csv

print("Début de la prédiction...")

limite_prediction = 1000 # nombre de lignes étudiées

k = 0
formes_input = []
formes_precises_probables = []
formes_principales_probables = []

# Lecture du fichier
df = pd.read_csv('logs.txt', sep="|", header=None, encoding='utf-8')
df2 = pd.read_csv("/content/drive/MyDrive/brouhaha_save/silence.txt", delimiter = "\t", header=None)

# Itération à travers les lignes
for index, row in df.iterrows():
    mot = row[0]
    id_ = row[1]
    idp = row[2]
    idpp = row[3]
    ide = row[4]
    idep = row[5]
    cv = row[6]
    freq = row[7]
    formes_input.append([mot, id_,  idp , idpp, ide , idep , cv, freq])
    k += 1

x = len(formes_input) - 1 # Nombre total de forme
p = 0

output = []
trouve = False
reduction = False
suite = False

# boucler sur chaque formes_input jusqu'à trouver une correspondance
while not trouve and p <= x:

    FiltreForme = 25
    partiel = False
    reduction = False
    recherche_suite = ""
    n = p
    for m in range(n, x + 1):
        recherche_suite = recherche_suite + " " + formes_input[m][4]
    print("Recherche de la suite:", recherche_suite, " avec le filtre:", FiltreForme)

    # continuer et réduire le filtre jusqu'à trouver une correspondance
    while partiel == False and FiltreForme > 3:

        print("Filtre actuel:", FiltreForme)
        df_recherche = pd.DataFrame()
        nombre_de_lignes = 0

        # rechercher uniquement les lignes qui contiennent la forme souhaitée
        forme_recherche = formes_input[0][4][:FiltreForme]
        df_recherche = df2[df2[0].str.contains(forme_recherche)]

        for i in range(1, x + 1):

            forme_recherche = formes_input[i][4][:FiltreForme]
            df_recherche = df_recherche[df_recherche[0].str.contains(forme_recherche)]

        nombre_de_lignes = df_recherche.shape[0]

        # si des lignes ont été trouvées, poursuivre l'évaluation
        if nombre_de_lignes > 0:

            # boucler sur l'ensemble des lignes filtrées et vérifier la correspondance de l'ordre des formes
            for index, row in df_recherche.head(limite_prediction).iterrows():
                doublon_precedent = ""
                ligne = row[0]
                # convertir la ligne en dataframe pandas
                mots = ligne.split("|")  # sépare la ligne en mots
                formes_par_mot = [mot.split(" & ") for mot in mots]
                df_result = pd.DataFrame(formes_par_mot).transpose()
                df_result = df_result.applymap(lambda x: x[:FiltreForme] if isinstance(x, str) else x)

                # rechercher la position de la première forme dans le tableau
                cherche = df_result.isin([formes_input[p][4][:FiltreForme]])
                colonnes = cherche.any()
                for colonne in colonnes.index:
                    if colonnes[colonne] and colonne + x + 1 - p <= df_result.shape[1]:
                        suite = True
                        # une fois la position de la première forme trouvée, vérifier la correspondance des colonnes suivantes
                        for i in range(1, x + 1 - p):
                            if colonne + i <= len(df_result.columns) and df_result[colonne + i].isin([formes_input[i][4][:FiltreForme]]).any():
                                suite = True
                                #print(f"La valeur '{mots_input[i][4][:FiltreForme]}' est trouvée dans la colonne {colonne + i} de la ligne {index}")
                            else:
                                suite = False
                                break

                        # si la suite est conforme, relever la forme suivante potentielle
                        if suite == True and colonne + x + 1 - p < len(df_result.columns):
                            partiel = True
                            formes_successives = df_result[colonne + x + 1 - p]
                            formes_successives = formes_successives.dropna()

                              # vérifier et mettre à jour les formes_principales_probables
                            for ligne_forme in formes_successives:
                                doublon = ligne_forme[:3]
                                forme_existe = False
                                if doublon != doublon_precedent:
                                    for i, row2 in enumerate(formes_principales_probables):
                                        if row2[0][:3] == doublon:
                                            formes_principales_probables[i][p + 1] += 1
                                            forme_existe = True
                                            break
                                    if not forme_existe:
                                        nouvelle_ligne = [doublon] + [0] * (x + 1)
                                        formes_principales_probables.append(nouvelle_ligne)
                                        formes_principales_probables[-1][p + 1] += 1
                                    print(nouvelle_ligne, df_result)
                                doublon_precedent = doublon


                            # Vérifier et mettre à jour les formes_precises_probables
                            if reduction == False:
                                trouve = True
                                for ligne_forme in formes_successives:
                                    forme_existe = False
                                    for i, row2 in enumerate(formes_precises_probables):
                                        if row2[0] == ligne_forme:
                                            formes_precises_probables[i][p + 1] += 1
                                            forme_existe = True
                                            break

                                    if not forme_existe:
                                        nouvelle_ligne = [ligne_forme] + [0] * (x + 1)
                                        formes_precises_probables.append(nouvelle_ligne)
                                        formes_precises_probables[-1][p + 1] += 1

        FiltreForme = FiltreForme - 1
        reduction = True
    p += 1

# calculer la forme la plus probable
sommes_colonne = []

if formes_precises_probables.count == 0:
    print("Erreur")

# Calcul de la moyenne des formes_principales_probables
# Calcul de la moyenne par colonne
for i, col in enumerate(zip(*formes_principales_probables)):
    # Si l'index de la colonne est supérieur ou égal à 1
    if i >= 1:
        # Somme des valeurs de la colonne et ajoute à la liste sum_cols
        sommes_colonne.append(sum(col))

# Initialisation de la liste pour stocker les moyennes de chaque colonne
moyennes_colonnes = []

# Itération sur chaque ligne de formes_precises_probables
for ligne in formes_principales_probables:
    # Initialisation de la nouvelle ligne pour stocker les moyennes
    nouvelle_ligne = [ligne[0]]  # on garde la forme telle quelle
    # Itération sur chaque colonne de la ligne, à partir de la deuxième colonne
    for i in range(1, len(ligne)):
        if sommes_colonne[i - 1] == 0:
            moyenne = 0
        else:
            # Calcul de la moyenne de la colonne
            moyenne = ligne[i] / sommes_colonne[i - 1]  # soustraction de 1 car sum_cols commence à l'index 0
            # Ajout de la moyenne à la nouvelle ligne
            nouvelle_ligne.append(moyenne)
    # Ajout de la nouvelle ligne aux moyennes
    moyennes_colonnes.append(nouvelle_ligne)

# Initialisation de la liste pour stocker les moyennes de chaque ligne
moyennes_ligne = []

# Itération sur chaque ligne de 'moyennes'
for ligne in moyennes_colonnes:
    # Calcul de la moyenne de chaque ligne (en excluant la première colonne qui contient la forme)
    moyenne_ligne = sum(ligne[1:]) / (len(ligne) - 1) # soustraction de 1 car on exclut la première colonne
    # Ajout de la moyenne de la ligne à la deuxième colonne
    ligne.insert(1, moyenne_ligne*100)
    # Ajout de la ligne modifiée à 'moyennes_ligne'
    moyennes_ligne.append(ligne)

# Tri de 'moyennes_ligne' par ordre décroissant en fonction de la deuxième colonne
moyennes_ligne = sorted(moyennes_ligne, key=lambda x: x[1], reverse=True)

best_forme_principale = moyennes_ligne[0][0]
best_score_principal = moyennes_ligne[0][1]

# Calcul de la moyenne des formes_precises_probables
sommes_colonne = []

formes_precises_probables = [ligne for ligne in formes_precises_probables if best_forme_principale in ligne[0][:3]]

# Calcul de la moyenne par colonne
for i, col in enumerate(zip(*formes_precises_probables)):
    # Si l'index de la colonne est supérieur ou égal à 1
    if i >= 1:
        # Somme des valeurs de la colonne et ajoute à la liste sum_cols
        sommes_colonne.append(sum(col))

# Initialisation de la liste pour stocker les moyennes de chaque colonne
moyennes_colonnes = []

# Itération sur chaque ligne de formes_precises_probables
for ligne in formes_precises_probables:
    nouvelle_ligne = [ligne[0]]
    # Itération sur chaque colonne de la ligne, à partir de la deuxième colonne
    for i in range(1, len(ligne)):
        if sommes_colonne[i - 1] == 0:
            moyenne = 0
        else:
            # Calcul de la moyenne de la colonne
            moyenne = ligne[i] / sommes_colonne[i - 1] # soustraction de 1 car sum_cols commence à l'index 0
            # Ajout de la moyenne à la nouvelle ligne
            nouvelle_ligne.append(moyenne)
    # Ajout de la nouvelle ligne aux moyennes
    moyennes_colonnes.append(nouvelle_ligne)

# Initialisation de la liste pour stocker les moyennes de chaque ligne
moyennes_ligne = []

# Itération sur chaque ligne de 'moyennes'
for ligne in moyennes_colonnes:
    # Calcul de la moyenne de chaque ligne (en excluant la première colonne qui contient la forme)
    moyenne_ligne = sum(ligne[1:]) / (len(ligne) - 1)  # soustraction de 1 car on exclut la première colonne
    # Ajout de la moyenne de la ligne à la deuxième colonne
    ligne.insert(1, moyenne_ligne*100)
    # Ajout de la ligne modifiée à 'moyennes_ligne'
    moyennes_ligne.append(ligne)

# Tri de 'moyennes_ligne' par ordre décroissant en fonction de la deuxième colonne
moyennes_ligne = sorted(moyennes_ligne, key=lambda x: x[1], reverse=True)

best_forme_precise = moyennes_ligne[0][0]
best_score_precis = moyennes_ligne[0][1]

nouvelle_ligne = ['[...]'] + [0] * (7)
formes_input.append(nouvelle_ligne)
formes_input[-1][1] = best_forme_precise
formes_input[-1][2] = best_forme_principale
formes_input[-1][3] = best_score_principal
formes_input[-1][4] = best_forme_precise
formes_input[-1][5] = best_score_precis
formes_input[-1][6] = 'XX'
formes_input[-1][7] = 0

with open('logs.txt', "w", encoding='utf-8', newline="") as fichier:
        writer = csv.writer(fichier, delimiter='|')
        writer.writerows(formes_input)

for line in formes_input:
    print(line)

if formes_input[-1][4] == "PON:po":
    print("Phrase terminée...")
else:
  print("Prédiction des formes terminée")

Début de la prédiction...
Recherche de la suite:  PRO:per:suj:m3s ADV:neg:ne VER:P0:T14:ind:pre:1s ADV:neg:pas PRO:int:cho  avec le filtre: 25
Filtre actuel: 25
Filtre actuel: 24
Filtre actuel: 23
Filtre actuel: 22
Filtre actuel: 21
Filtre actuel: 20
Filtre actuel: 19
Filtre actuel: 18
Filtre actuel: 17
Filtre actuel: 16
Filtre actuel: 15
Filtre actuel: 14
Filtre actuel: 13
Filtre actuel: 12
['PRO', 1, 0, 0, 0, 0]              0           1             2            3            4   \
0  PRO:per:suj:  ADV:neg:ne  VER:P0:T14:i  ADV:neg:pas  PRO:int:cho   

           5             6       7           8             9   ...  \
0  PRO:ind:ms  VER:S0:T13:s  PON:vi  PRO:per:ms  VER:S1:T13:s  ...   

             33            34       35            36            37  \
0  VER:X2:A9:in  VER:S0:a:T13  PRE:man  PRO:per:com:  NOM:psy:non:   

           38          39            40          41      42  
0  ADJ:psy:es  ART:par:fs  NOM:soc:non:  ADJ:log:fs  PON:po  

[1 rows x 43 columns]
Recherche 

#**3) Affichage du mot probable suivant**



In [31]:
import pandas as pd
import csv
import random

indice_creativite = 10

phrase = ""

mots_input= []

df = pd.read_csv('logs.txt', sep="|", header=None, encoding='utf-8')

# Itération à travers les lignes
for index, row in df.iterrows():
    mot = row[0]
    id_ = row[1]
    idp = row[2]
    idpp = row[3]
    ide = row[4]
    idep = row[5]
    cv = row[6]
    freq = row[7]

    mots_input.append([mot, id_,  idp , idpp, ide , idep , cv, freq])

forme_principale = mots_input[-1][2]
pourcentage_forme_principale = mots_input[-1][3]
forme_probable = mots_input[-1][4]
pourcentage_forme_probable = mots_input[-1][5]
CV = ""

if mots_input[-2][6][-1] == "V":
  CV = "C"
else:
  CV = "V"

# lire le fichier
with open('/content/drive/MyDrive/brouhaha_save/dictionnaire.txt', 'r', encoding="utf-8") as file2:
    reader = csv.reader(file2, delimiter='|')

    # initialiser une liste pour stocker les lignes qui correspondent
    mots_probables = []

    # parcourir chaque ligne du Dictionnaire
    for row in reader:
        if forme_probable in row[1]:
            # vérifier si le mot probable doit commencer par une consonne
            if CV == "C":
                if row[2][1] == "C":
                    mots_probables.append(row)
            else:
                mots_probables.append(row)

# si aucun résultat trouvé, rechercher sans filtre consonne/voyelle
if len(mots_probables) == 0:
    with open('/content/drive/MyDrive/brouhaha_save/dictionnaire.txt', 'r', encoding="utf-8") as file2:
        reader = csv.reader(file2, delimiter='|')
        # initialiser une liste pour stocker les lignes qui correspondent
        mots_probables = []

        # parcourir chaque ligne du Dictionnaire
        for row in reader:
            if forme_probable in row[1]:
                mots_probables.append(row)

mots_probables = sorted(mots_probables, key=lambda row: row[6], reverse=True)

x = random.randint(1, indice_creativite)

if x > len(mots_probables):
    x = len(mots_probables)

mot_choisi = mots_probables[x - 1][0]
formes_probables = mots_probables[x - 1][1]
cv_choisi = mots_probables[x - 1][2]
frequence = mots_probables[x - 1][6]

# vérifier si le mot choisit et le mot précédent sont compatible
if cv_choisi[-2][0][-1] == "V" and CV == "C":
    if mots_input[-2][0] == "ce":
        mots_input[-2][0] = mots_input[-2][0].replace("ce", "c'")
    if mots_input[-2][0] == "je":
        mots_input[-2][0] = mots_input[-2][0].replace("je", "j'")
    if mots_input[-2][0] == "ne":
        mots_input[-2][0] = mots_input[-2][0].replace("ne", "n'")
    if mots_input[-2][0] == "se":
        mots_input[-2][0] = mots_input[-2][0].replace("se", "s'")
    if mots_input[-2][0] == "te":
        mots_input[-2][0] = mots_input[-2][0].replace("te", "t'")
    if mots_input[-2][0] == "que":
        mots_input[-2][0] = mots_input[-2][0].replace("que", "qu'")
    if mots_input[-2][0] == "me":
        mots_input[-2][0] = mots_input[-2][0].replace("me", "m'")
    if mots_input[-2][0] == "de":
        mots_input[-2][0] = mots_input[-2][0].replace("de", "d'")
elif (cv_choisi[-2][0][-1] == "'" or cv_choisi[-2][0][-1] == "C") and CV == "V":
    if mots_input[-2][0] == "c'":
        mots_input[-2][0] = mots_input[-2][0].replace("c'", "ce")
    if mots_input[-2][0] == "j'":
        mots_input[-2][0] = mots_input[-2][0].replace("j'", "je")
    if mots_input[-2][0] == "n'":
        mots_input[-2][0] = mots_input[-2][0].replace("n'", "ne")
    if mots_input[-2][0] == "s'":
        mots_input[-2][0] = mots_input[-2][0].replace("se", "s'")
    if mots_input[-2][0] == "t'":
        mots_input[-2][0] = mots_input[-2][0].replace("t'", "te")
    if mots_input[-2][0] == "qu'":
        mots_input[-2][0] = mots_input[-2][0].replace("qu'", "que")
    if mots_input[-2][0] == "m'":
        mots_input[-2][0] = mots_input[-2][0].replace("m'", "me")
    if mots_input[-2][0] == "d'":
        mots_input[-2][0] = mots_input[-2][0].replace("d'", "de")

# supprimer la dernière ligne de la liste pour la remplacer par le mot choisi
mots_input.pop()

nouvelle_ligne = [mot_choisi] + [0] * (7)
mots_input.append(nouvelle_ligne)
mots_input[-1][1] = formes_probables
mots_input[-1][2] = forme_principale
mots_input[-1][3] = pourcentage_forme_principale
mots_input[-1][4] = forme_probable
mots_input[-1][5] = pourcentage_forme_probable
mots_input[-1][6] = cv_choisi
mots_input[-1][7] = frequence

with open('logs.txt', "w", encoding='utf-8', newline="") as fichier:
        writer = csv.writer(fichier, delimiter='|')
        writer.writerows(mots_input)

for line in mots_input:
    #print(line)
    phrase = " ".join(sublist[0] for sublist in mots_input)

phrase = phrase.replace(" , ", ",")
phrase = phrase.replace("' ", "'")
phrase = phrase.replace(" .", ".")

print(phrase)

il ne pense pas qu'il
